# **추천시스템**
## **sklearn의 surprise 모듈을 사용**
웹을 위한 머신러닝을 surprise로 구현

<br>
## **1 추천시스템의 이해**
Item이 많고, Query를 잘 모를때 유용하다
1. Popularity, High Rated Based (가장단순) : 높은 평점의 item을 추천, 결과값이 동일한 단점이 있다
1. Collaborative Filtering (중간) : user & item의 rating을 이용하여, 사용자/item 유사도를 찾는다
    1. 즉, 해당 user가 해당 item을 얼마나 좋아할 것인지 수치적으로 예측하는 것으로 User-based CF, Item-based CF라고도 함
    1. Similarity 계산은 Euclidean distance, cosine, pearson 등 여러 수학적 유사도를 측정하여, 평점을 weighted sum으로 계산하여 평점을 예측
    1. Personalization (개인화)는 각 개인의 성향에 맞는 item을 추천

<br>
## **2 간단한 추천시스템의 구현**
### **1) Popularity, High Rated Based**

In [57]:
ratings={ 'Dave':{'달콤한인생':5,'범죄도시':3,'샤인':3},
          'David':{'달콤한인생':5,'범죄도시':1,'샤인':4},
          'Alex':{'달콤한인생':0,'범죄도시':4,'샤인':5},
          'Andy':{'달콤한인생':2,'범죄도시':1,'샤인':5} }

In [58]:
# 가장 평점이 높은 순서로 추천한다
movie_dict = dict()
for rating in ratings:
    for movie in ratings[rating].keys():
        if movie not in movie_dict:
            movie_dict[movie] = ratings[rating][movie]
        else:
            movie_dict[movie] = (movie_dict[movie] + ratings[rating][movie]) 

for movie in ratings[rating].keys():
    movie_dict[movie] = movie_dict[movie] / 4

import operator
sorted_x = sorted(movie_dict.items(), key=operator.itemgetter(1), reverse=True)

print(sorted_x[:2])

[('샤인', 4.25), ('달콤한인생', 3.0)]


<br>
### **2) Collaborative Filtering**
1. 사용자/아이템 - 항목 선호도(평점) 행렬을 사용하여 사용자/아이템들 간의 유사도를 계산한다.
1. 사용자/아이템 간의 모든 항목에 대해 예측 값을 계산하고, 상위 N개의 추천 목록을 생성한다.

In [59]:
# 'Alex'와 다른 사용가간의 '범죄도시'와 '샤인'유사도 측정 
# 피타고라스 정리로 유사도 측정
import math
def sim(i, j):
    return math.sqrt(pow(i,2)+pow(j,2))

for i in ratings:
    if i != 'Alex':
        num1 = ratings.get('Alex').get('범죄도시') - ratings.get(i).get('범죄도시')
        num2 = ratings.get('Alex').get('샤인') - ratings.get(i).get('샤인')
        print(i," : ", sim(num1,num2))

Dave  :  2.23606797749979
David  :  3.1622776601683795
Andy  :  3.0


In [60]:
# 'Alex'의 유사도 측정 결과값을 0~1로 정규화
for i in ratings:
    if i != 'Alex':
        num1 = ratings.get('Alex').get('범죄도시') - ratings.get(i).get('범죄도시')
        num2 = ratings.get('Alex').get('샤인') - ratings.get(i).get('샤인')
        print(i," : ", 1 / ( 1 + sim(num1,num2) ) )

Dave  :  0.3090169943749474
David  :  0.2402530733520421
Andy  :  0.25


In [61]:
# 'Dave'의 유사도 측정 결과값을 0~1로 정규화
for i in ratings:
    if i != 'Dave':
        num1 = ratings.get('Dave').get('범죄도시') - ratings.get(i).get('범죄도시')
        num2 = ratings.get('Dave').get('샤인') - ratings.get(i).get('샤인')
        print(i," : ", 1 / ( 1 + sim(num1,num2) ) )

David  :  0.3090169943749474
Alex  :  0.3090169943749474
Andy  :  0.2612038749637414


<br>
## **3 유사도(Similarity) 측정**
### **1) Mean Squared Difference Similarity (평균제곱을 활용)**
User-based Collaborative Filter ,Item-based Collaborative Filter
$$ msd (u(사용자1)와  v(사용자2)간의 거리) = \frac{(u 와 v 평가상품의 평점차의 제곱)}{(u와 v 평가상품의 수)} $$
$$ msd\_sim (유사도) = \frac{1}{msd(u,v) + 1} $$

In [62]:
# 1 Mean Squared Difference Similarity
# Mean Squared Difference (msd) 의 역수를 계산하여 차이가 클 수록 Similarity 값은 작아진다!
# MSD가 0이 되는 경우를 대응하기 위해 1을 무조건 더해준다

def sim_msd(data, name1, name2):
    sum = 0
    count = 0
    for movies in data[name1]:
        if movies in data[name2]: #같은 영화를 봤다면
            sum += pow(data[name1][movies] - data[name2][movies], 2)
            count += 1

    sim_msd = 1 / ( 1 + (sum / count) )
    return round(sim_msd, 4)

sim_msd(ratings, 'Dave','Alex')

0.0909

<br>
### **2) Cosine Similarity (코사인 유사도)**
1. Cosine Similarity에서 **"−1은"** 완전히 반대, **"0은"** 서로 독립, **"1은"** 완전히 같은 경우를 의미
1. 두 벡터간의 유사도를 cosine으로 계산한다
1. u(사용자1) 과 v(사용자2)간의 유사도 측정시 **모든 상품 평점의 기하평균값**으로 계산한다
$$ u \bullet v = |u| \bullet |v| \cos \theta  \ggg  \cos \theta = \frac {u \bullet v}{|u| \bullet |v|} $$
$$ \cos 유사도 = \frac {u \bullet v}{|u| \bullet |v|} $$


In [63]:
import math
def sim_cosine(data, name1, name2):
    sum_name1, sum_name2, sum_name1_name2, count = 0,0,0,0
    for movies in data[name1]:
        if movies in data[name2]: #같은 영화를 봤다면
            sum_name1 += pow(data[name1][movies], 2)
            sum_name2 += pow(data[name2][movies], 2)
            sum_name1_name2 += data[name1][movies] * data[name2][movies]
    sim_cosine = sum_name1_name2 / (math.sqrt(sum_name1) * math.sqrt(sum_name2))
    return round(sim_cosine, 4)

# Dave','Alex' 간의 cosine 유사도
sim_cosine(ratings, 'Dave','Alex')

0.643

<br>
### **3)Pearson Similarity (피어슨 유사도)**
두 벡터의 상관계수(Pearson correlation coefficient)로써 **유사도가 가장 높을 경우 값이 1, *가장 낮을 경우 -1의 값**을 갖는다

In [64]:
def sim_pearson(data, name1, name2):
    avg_name1, avg_name2 ,count = 0, 0, 0
    for movies in data[name1]:
        if movies in data[name2]: #같은 영화를 봤다면
            avg_name1 = data[name1][movies]
            avg_name2 = data[name2][movies]
            count += 1

    avg_name1 = avg_name1 / count
    avg_name2 = avg_name2 / count

    sum_name1, sum_name2, sum_name1_name2, count = 0,0,0,0
    for movies in data[name1]:
        if movies in data[name2]: #같은 영화를 봤다면
            sum_name1 += pow(data[name1][movies] - avg_name1, 2)
            sum_name2 += pow(data[name2][movies] - avg_name2, 2)
            sum_name1_name2 += (data[name1][movies] - avg_name1) * (data[name2][movies] - avg_name2)

    sim_pearson = sum_name1_name2 / (math.sqrt(sum_name1)*math.sqrt(sum_name2))
    return round(sim_pearson, 4)

sim_pearson(ratings, 'Dave','Alex')

0.2166

<br>
### **4) 전체 유사도 측정**
top_match

In [65]:
def top_match(data, name, index=3, sim_function=sim_pearson):
    li=[]
    for i in data: #딕셔너리를 돌고
        if name!=i: #자기 자신이 아닐때만
            li.append((sim_function(data,name,i),i)) #sim_function()을 통해 상관계수를 구하고 li[]에 추가
    li.sort() #오름차순
    li.reverse() #내림차순
    return li[:index]

top_match(ratings, 'Dave', 3)

[(0.8681, 'David'), (0.3984, 'Andy'), (0.2166, 'Alex')]

In [66]:
print('sim_msd \n {} \nsim_cosine \n {}\nsim_pearson \n{}'.format(
    top_match(ratings, 'Dave', 3, sim_function=sim_msd),
    top_match(ratings, 'Dave', 3, sim_function=sim_cosine),
    top_match(ratings, 'Dave', 3, sim_function=sim_pearson)))

sim_msd 
 [(0.375, 'David'), (0.15, 'Andy'), (0.0909, 'Alex')] 
sim_cosine 
 [(0.9412, 'David'), (0.7796, 'Andy'), (0.643, 'Alex')]
sim_pearson 
[(0.8681, 'David'), (0.3984, 'Andy'), (0.2166, 'Alex')]


<br>
## **4 KNN을 활용한 예측값 및 유사도(Similarity) 측정**
### **1) K Nearest Neighbors(KNN) 가중치 예측 기법**
User-based Collaborative Filter ,Item-based Collaborative Filter
$$ msd (u(사용자1)와  v(사용자2)간의 거리) = \frac{(u 와 v 평가상품의 평점차의 제곱)}{(u와 v 평가상품의 수)} $$
$$ msd\_sim (유사도) = \frac{1}{msd(u,v) + 1} $$

In [67]:
ratings_expand = {
    '마동석': {'택시운전사': 3.5,'남한산성': 1.5,'킹스맨:골든서클': 3.0,'범죄도시': 3.5,'아이 캔 스피크': 2.5,'꾼': 3.0,},
    '이정재': {'택시운전사': 5.0,'남한산성': 4.5,'킹스맨:골든서클': 0.5,'범죄도시': 1.5,'아이 캔 스피크': 4.5,'꾼': 5.0,},
    '윤계상': {'택시운전사': 3.0,'남한산성': 2.5,'킹스맨:골든서클': 1.5,'범죄도시': 3.0,'아이 캔 스피크': 3.5,'꾼': 3.0,},
    '설경구': {'택시운전사': 2.5,'남한산성': 3.0,'범죄도시': 4.5,'꾼': 4.0,},
    '최홍만': {'남한산성': 4.5,'킹스맨:골든서클': 3.0,'꾼': 4.5,'범죄도시': 3.0,'아이 캔 스피크': 2.5,},
    '홍수환': {'택시운전사': 3.0,'남한산성': 4.0,'킹스맨:골든서클': 1.0,'범죄도시': 3.0,'꾼': 3.5,'아이 캔 스피크': 2.0,},
    '나원탁': {'택시운전사': 3.0,'남한산성': 4.0,'꾼': 3.0,'범죄도시': 5.0,'아이 캔 스피크': 3.5,},
    '소이현': {'남한산성': 4.5,'아이 캔 스피크': 1.0,'범죄도시': 4.0} }

In [74]:
# KNN With Means : 평점들을 평균값 기준으로 가중 평균한다.
for name in ratings_expand:
    sum = 0
    count = 0
    for movies in ratings_expand[name]:
        sum += ratings_expand[name][movies]
        count += 1
    ratings_expand[name]['avg'] = sum / count
    
# ratings_expand

In [68]:
# KNN을 활용한 추천시스템
def getRecommendation (data, person, k=3, sim_function=sim_pearson):
    result = top_match(data, person, k)
    # score : 평점합, li : 리턴값, score_dic : 유사도 총합, sim_dic : 평점총합
    score, li, score_dic, sim_dic = 0, list(), dict(), dict() 
    for sim, name in result:                     # 튜플이므로 한번에
        print(sim, name)
        if sim < 0 : continue                    # 유사도가 양수인 사람만 추출

        for movie in data[name]: 
            if movie not in data[person]:        # name이 평가를 내리지 않은 영화
                score += sim * data[name][movie] # 그사람의 영화평점 * 유사도
                score_dic.setdefault(movie, 0)   # 기본값 설정
                score_dic[movie] += score        # 합계
                sim_dic.setdefault(movie, 0)     # 조건에 맞는 유사도의 누적합
                sim_dic[movie] += sim
            score = 0                            # 다른 영화 측정을 위해서 값을 초기화

    for key in score_dic: 
        score_dic[key] = score_dic[key] / sim_dic[key] # 평점 총합/ 유사도 총합
        li.append((score_dic[key],key))                # list((tuple))의 리턴을 위해서.

    li.sort()                                          # 오름차순
    li.reverse()                                       # 내림차순
    return li

In [69]:
# '소이현'과 유사한 사용자는?
getRecommendation(ratings_expand, '소이현')

0.9331 홍수환
0.891 최홍만
0.8452 나원탁


[(3.6754767167422173, '꾼'),
 (2.9999999999999996, '택시운전사'),
 (1.9769201249931474, '킹스맨:골든서클')]

In [72]:
# '최홍만' 과 가장 유사한 사용자는?
# 단 유사도 함수는sim_cosine, k는 2를 사용 
getRecommendation(ratings_expand, '최홍만', k=3, sim_function=sim_cosine)

0.9608 홍수환
0.9518 소이현
0.9064 나원탁


[(2.9999999999999996, '택시운전사')]

<br>
## **5 Latent Factor 모형 (PCA와 Latent Factor 모형)**
Latent Factor 모형은 복잡한 사용자/영화 특성을 몇 개의 벡터로 간략화해보겠다는 모형

# http://www.fun-coding.org/recommend_basic5.html

<br>
## **1 Utility Marix 구현**
유틸리티 매트릭스 구현

In [1]:
# ! pip install surprise
# 영화 데이터 불러오기 [ 사용자, 영화, 평점 데이터 ]
import surprise
import numpy as np
import pandas as pd

data = surprise.Dataset.load_builtin('ml-100k')
df   = pd.DataFrame(data.raw_ratings, 
                    columns = ["user", "item", "rate", "id"])
del df["id"]; print(df.shape); df.head(5)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /home/markbaum/.surprise_data/ml-100k
(100000, 3)


,user,item,rate
0,196,242,3.0
1,186,302,3.0
2,22,377,1.0
3,244,51,2.0
4,166,346,1.0


In [4]:
# Pivot Table 로  평점 행렬(rate matrix) R 을 만든다
# y축 : 아이디("user") , x축 : 영화("item") 
df_table      = df.set_index(["user", "item"]).unstack()
df_table_temp = df_table.iloc[212:218, 808:817]
df_table_temp.fillna("")

rate                                
item  211 212 213 214 215 216 217 218 219
user                                     
290     3                   4       2    
291         4       4   4           4   4
292                 3                    
293     4       3       4   4   3   2    
294                                      
295             5       5   5   4   5

In [ ]:
# Warm start, cold start 여부를 판단
# % matplotlib inline
# # df_table 데이터 분포를 시각적 분석
# import matplotlib.pyplot as plt
# plt.imshow(df_table);     plt.grid(False)
# plt.xlabel("item");       plt.ylabel("user")
# plt.title("Rate Matrix"); plt.show()

<br>
## **2 추천시스템 구현하기**
k-nn으로 

In [5]:
data.split(n_folds=3)
sim_options = {'name': 'pearson'}    # 피어슨 유사도 측정
algo = surprise.KNNBasic(sim_options = sim_options)
surprise.model_selection.cross_validate(algo, data)

Computing the pearson similarity matrix...
Computing the pearson similarity matrix...
Computing the pearson similarity matrix...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


{'fit_time': (2.217965841293335,
  2.40482497215271,
  2.762643814086914,
  2.276320219039917,
  1.6477456092834473),
 'test_mae': array([0.80623872, 0.79886829, 0.8053499 , 0.80413584, 0.79560719]),
 'test_rmse': array([1.01312409, 1.00656244, 1.01404426, 1.01511344, 1.00315338]),
 'test_time': (4.183986663818359,
  4.222134351730347,
  4.362393379211426,
  4.569180488586426,
  2.3392112255096436)}